In [1]:
import pandas as pd
import re
from collections import namedtuple
import pdfplumber
import os 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np

### 1. Importing and  merging pop files
### 2. Adding Metropolitans and Micropolitans

In [2]:
pop_2010_2019 = pd.read_excel('../data/raw_data/NC/pop_est_2010-2019_nc.xlsx',sheet_name='Sheet1')
pop_2010_2019 = pd.DataFrame(pop_2010_2019)
pop_2010_2019 = pop_2010_2019.drop(['Census','Estimates Base',2010,2011,2012],axis=1)
pop_2010_2019.head()

,County,2013,2014,2015,2016,2017,2018,2019
0,Alamance,154162,155384,157037,160283,163276,166514,169509
1,Alexander,37012,36987,36960,37133,37143,37331,37497
2,Alleghany,10903,10891,10856,10919,10988,11138,11137
3,Anson,25957,25965,25602,25165,24829,24470,24446
4,Ashe,26840,26764,26628,26600,26787,27099,27203


In [3]:
pop_2020_2021 = pd.read_excel('../data/raw_data/NC/pop_est_2020-2021_nc.xlsx',sheet_name='Sheet1')
pop_2020_2021 = pd.DataFrame(pop_2020_2021)
pop_2020_2021 = pop_2020_2021.drop('Estimates Base',axis=1)
pop_2020_2021.head()

,County,2020,2021
0,Alamance,171842,173877
1,Alexander,36430,36644
2,Alleghany,10899,11049
3,Anson,22030,22060
4,Ashe,26569,26711


In [4]:
pop_combined = pd.merge(pop_2010_2019,pop_2020_2021,how="left",on="County")
pop_combined['County'] = pop_combined['County'].str.strip()
pop_combined['County'] = pop_combined['County'].str.upper()
pop_combined.head()

,County,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,ALAMANCE,154162,155384,157037,160283,163276,166514,169509,171842,173877
1,ALEXANDER,37012,36987,36960,37133,37143,37331,37497,36430,36644
2,ALLEGHANY,10903,10891,10856,10919,10988,11138,11137,10899,11049
3,ANSON,25957,25965,25602,25165,24829,24470,24446,22030,22060
4,ASHE,26840,26764,26628,26600,26787,27099,27203,26569,26711


In [5]:
pop_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   County  100 non-null    object
 1   2013    100 non-null    int64 
 2   2014    100 non-null    int64 
 3   2015    100 non-null    int64 
 4   2016    100 non-null    int64 
 5   2017    100 non-null    int64 
 6   2018    100 non-null    int64 
 7   2019    100 non-null    int64 
 8   2020    100 non-null    int64 
 9   2021    100 non-null    int64 
dtypes: int64(9), object(1)
memory usage: 8.6+ KB


In [6]:
metros = pd.read_excel('../data/raw_data/metro_areas.xlsx',sheet_name='Filtered_States')
metros_df = pd.DataFrame(metros)
metros_nc = metros_df[metros_df['State Name']=='North Carolina']
metros_nc['County'] = metros_nc['County'].str.strip()
metros_nc['County'] = metros_nc['County'].str.upper()
metros_nc.head()

C:\Users\iulia\AppData\Local\Temp/ipykernel_30504/1598154956.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metros_nc['County'] = metros_nc['County'].str.strip()
C:\Users\iulia\AppData\Local\Temp/ipykernel_30504/1598154956.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metros_nc['County'] = metros_nc['County'].str.upper()


,CBSA Title,Metropolitan/Micropolitan Statistical Area,County,State Name
1,"Burlington, NC",Metropolitan Statistical Area,ALAMANCE,North Carolina
2,"Hickory-Lenoir-Morganton, NC",Metropolitan Statistical Area,ALEXANDER,North Carolina
7,"Charlotte-Concord-Gastonia, NC-SC",Metropolitan Statistical Area,ANSON,North Carolina
21,"Washington, NC",Micropolitan Statistical Area,BEAUFORT,North Carolina
38,"Myrtle Beach-Conway-North Myrtle Beach, SC-NC",Metropolitan Statistical Area,BRUNSWICK,North Carolina


### Extracting data for years 2013 thru 2016 and combining with pop along with Metropolitans and Micropolitans

In [7]:
file_2013_2016 ='../data/raw_data/NC/NC_2012_2016.pdf'

In [8]:
pdf = pdfplumber.open(file_2013_2016)
txt = pdf.pages[55].extract_text()
txt = txt.split("\n")
new_list = txt[0:7]
new_list

['Caldwell County',
 '2012 2013 2014 2015 2016 5 Year Avg.',
 'Reportable Crashes Injuries Crashes Injuries Crashes Injuries Crashes Injuries Crashes Injuries Crashes Injuries',
 'Fatal 5 7 8 8 8 10 12 12 9 10 8 9',
 'Non Fatal Injury 524 779 528 835 533 847 584 925 591 860 552 849',
 'PDO 994 999 1,097 1,140 1,159 1,078',
 'Total 1,523 786 1,535 843 1,638 857 1,736 937 1,759 870 1,638 859']

In [9]:
county = new_list[0]
county = county.replace("County","")
county = county.strip()
fatalities = new_list[3].split(" ")
del fatalities[0:3]
del fatalities[-2:]
fatalities = list(np.int_(fatalities))
total = new_list[6].replace(",","")
total = total.split(" ")
del total[0:3]
del total[-2:]
total = list(np.int_(total))
print(county,fatalities,total)    

Caldwell [8, 8, 8, 10, 12, 12, 9, 10] [1535, 843, 1638, 857, 1736, 937, 1759, 870]


In [10]:
my_dic_2013 = {}
my_dic_2013['County'] = county
my_dic_2013['fatalities'] = fatalities[0]
my_dic_2013['injuries'] = total[1]
my_dic_2013['total_crashes'] = total[0]
print(my_dic_2013) 

{'County': 'Caldwell', 'fatalities': 8, 'injuries': 843, 'total_crashes': 1535}


In [11]:
my_dic_2014 = {}
my_dic_2014['County'] = county
my_dic_2014['fatalities'] = fatalities[2]
my_dic_2014['injuries'] = total[3]
my_dic_2014['total_crashes'] = total[2]
print(my_dic_2014)

{'County': 'Caldwell', 'fatalities': 8, 'injuries': 857, 'total_crashes': 1638}


In [12]:
my_dic_2015 = {}
my_dic_2015['County'] = county
my_dic_2015['fatalities'] = fatalities[4]
my_dic_2015['injuries'] = total[5]
my_dic_2015['total_crashes'] = total[4]
print(my_dic_2015)

{'County': 'Caldwell', 'fatalities': 12, 'injuries': 937, 'total_crashes': 1736}


In [13]:
nc_l_2013 = []
nc_l_2014 = []
nc_l_2015 = []
pages = list(range(42,142))
pdf = pdfplumber.open(file_2013_2016)
for page in pages:
    my_dic_2013 = {}
    my_dic_2014 = {}    
    my_dic_2015 = {}
    txt =pdf.pages[page].extract_text()
    txt = txt.split("\n")
    new_list = txt[0:7]
    county = new_list[0]
    county = county.replace("County","")
    county = county.strip()
    fatalities = new_list[3].split(" ")
    del fatalities[0:3]
    del fatalities[-2:]
    fatalities = list(np.int_(fatalities))
    total = new_list[6].replace(",","")
    total = total.split(" ")
    del total[0:3]
    del total[-2:]
    total = list(np.int_(total))
    my_dic_2013['County'] = county
    my_dic_2013['fatalities'] = fatalities[1]
    my_dic_2013['injuries'] = total[1]
    my_dic_2013['total_crashes'] = total[0]
    nc_l_2013.append(my_dic_2013)
    my_dic_2014['County'] = county
    my_dic_2014['fatalities'] = fatalities[3]
    my_dic_2014['injuries'] = total[3]
    my_dic_2014['total_crashes'] = total[2]
    nc_l_2014.append(my_dic_2014)
    my_dic_2015['County'] = county
    my_dic_2015['fatalities'] = fatalities[4]
    my_dic_2015['injuries'] = total[5]
    my_dic_2015['total_crashes'] = total[4]    
    nc_l_2015.append(my_dic_2015)  
nc_df_2013 = pd.DataFrame(nc_l_2013)
nc_df_2014 = pd.DataFrame(nc_l_2014)
nc_df_2015 = pd.DataFrame(nc_l_2015)
nc_df_2013 ['County'] = nc_df_2013 ['County'].str.strip()
nc_df_2013 ['County'] = nc_df_2013 ['County'].str.upper()
nc_df_2014 ['County'] = nc_df_2014 ['County'].str.strip()
nc_df_2014 ['County'] = nc_df_2014 ['County'].str.upper()
nc_df_2015 ['County'] = nc_df_2015 ['County'].str.strip()
nc_df_2015 ['County'] = nc_df_2015 ['County'].str.upper()
df_pop_nc13 = pd.merge(nc_df_2013,pop_combined[['County', 2013]],on="County",how='left')
df_pop_nc14 = pd.merge(nc_df_2014,pop_combined[['County', 2014]],on="County",how='left')
df_pop_nc15 = pd.merge(nc_df_2015,pop_combined[['County', 2015]],on="County",how='left')
df_comb_nc13 = pd.merge(df_pop_nc13,metros_nc[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_comb_nc14 = pd.merge(df_pop_nc14,metros_nc[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_comb_nc15 = pd.merge(df_pop_nc15,metros_nc[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_comb_nc13.rename(columns = {2013:'population'}, inplace = True)
df_comb_nc13.insert(0,'Year',2013)
df_comb_nc14.rename(columns = {2014:'population'}, inplace = True)
df_comb_nc14.insert(0,'Year',2014)
df_comb_nc15.rename(columns = {2015:'population'}, inplace = True)
df_comb_nc15.insert(0,'Year',2015)
df_comb_nc13.to_excel('../data/cleaned_data/2013_NC.xlsx', engine='xlsxwriter',index=False)
df_comb_nc14.to_excel('../data/cleaned_data/2014_NC.xlsx', engine='xlsxwriter',index=False)
df_comb_nc15.to_excel('../data/cleaned_data/2015_NC.xlsx', engine='xlsxwriter',index=False)

### Extracting data for years 2017 thru 2020 and combining with pop along with Metropolitans and Micropolitans

In [14]:
file_2017_2020 ='../data/raw_data/NC/NC_2016_2020.pdf'

In [15]:
pdf = pdfplumber.open(file_2017_2020)
txt = pdf.pages[56].extract_text()
txt = txt.split("\n")
new_list = txt[0:7]
new_list

['Camden County',
 '2016 2017 2018 2019 2020 5 Year Avg.',
 'Reportable Crashes Injuries Crashes Injuries Crashes Injuries Crashes Injuries Crashes Injuries Crashes Injuries',
 'Fatal 0 0 3 3 3 5 0 0 2 2 2 2',
 'Non Fatal Injury 53 80 37 46 41 59 42 62 34 51 41 60',
 'PDO 111 113 143 153 109 126',
 'Total 164 80 153 49 187 64 195 62 145 53 169 62']

In [16]:
county = new_list[0]
county = county.replace("County","")
county = county.strip()
fatalities = new_list[3].split(" ")
del fatalities[0]
del fatalities[-2:]
total = new_list[6].split(" ")
del total[0]
del total[-2:]
print(county,fatalities,total)

Camden ['0', '0', '3', '3', '3', '5', '0', '0', '2', '2'] ['164', '80', '153', '49', '187', '64', '195', '62', '145', '53']


In [17]:
my_dic_2016 = {}
my_dic_2016['County'] = county
my_dic_2016['fatalities'] = fatalities[0]
my_dic_2016['injuries'] = total[1]
my_dic_2016['total_crashes'] = total[0]
print(my_dic_2016)

{'County': 'Camden', 'fatalities': '0', 'injuries': '80', 'total_crashes': '164'}


In [18]:
nc_l_2016 = []
nc_l_2017 = []
nc_l_2018 = []
nc_l_2019 = []
nc_l_2020 = []
pages = list(range(42,142))
pdf = pdfplumber.open(file_2017_2020)
for page in pages:
    my_dic_2016 = {}
    my_dic_2017 = {}
    my_dic_2018 = {}
    my_dic_2019 = {}
    my_dic_2020 = {}
    txt =pdf.pages[page].extract_text()
    txt = txt.split("\n")
    new_list = txt[0:7]
    county = new_list[0]
    county = county.replace("County","")
    county = county.strip()
    fatalities = new_list[3].split(" ")
    del fatalities[0]
    del fatalities[-2:]
    total = new_list[6].split(" ")
    del total[0]
    del total[-2:]
    my_dic_2016['County'] = county
    my_dic_2016['fatalities'] = fatalities[0]
    my_dic_2016['injuries'] = total[1]
    my_dic_2016['total_crashes'] = total[0]
    nc_l_2016.append(my_dic_2016)
    my_dic_2017['County'] = county
    my_dic_2017['fatalities'] = fatalities[2]
    my_dic_2017['injuries'] = total[3]
    my_dic_2017['total_crashes'] = total[2]
    nc_l_2017.append(my_dic_2017)
    my_dic_2018['County'] = county
    my_dic_2018['fatalities'] = fatalities[4]
    my_dic_2018['injuries'] = total[5]
    my_dic_2018['total_crashes'] = total[4]
    nc_l_2018.append(my_dic_2018)
    my_dic_2019['County'] = county
    my_dic_2019['fatalities'] = fatalities[6]
    my_dic_2019['injuries'] = total[7]
    my_dic_2019['total_crashes'] = total[6]
    nc_l_2019.append(my_dic_2019)
    my_dic_2020['County'] = county
    my_dic_2020['fatalities'] = fatalities[8]
    my_dic_2020['injuries'] = total[9]
    my_dic_2020['total_crashes'] = total[7]
    nc_l_2020.append(my_dic_2020)
nc_df_2016 = pd.DataFrame(nc_l_2016)    
nc_df_2017 = pd.DataFrame(nc_l_2017)
nc_df_2018 = pd.DataFrame(nc_l_2018)
nc_df_2019 = pd.DataFrame(nc_l_2019) 
nc_df_2020 = pd.DataFrame(nc_l_2020)
nc_df_2016 ['County'] = nc_df_2016 ['County'].str.strip()
nc_df_2016 ['County'] = nc_df_2016 ['County'].str.upper()
nc_df_2017 ['County'] = nc_df_2017 ['County'].str.strip()
nc_df_2017 ['County'] = nc_df_2017 ['County'].str.upper()
nc_df_2018 ['County'] = nc_df_2018 ['County'].str.strip()
nc_df_2018 ['County'] = nc_df_2018 ['County'].str.upper()
nc_df_2019 ['County'] = nc_df_2019 ['County'].str.strip()
nc_df_2019 ['County'] = nc_df_2019 ['County'].str.upper()
nc_df_2020 ['County'] = nc_df_2020 ['County'].str.strip()
nc_df_2020 ['County'] = nc_df_2020 ['County'].str.upper()
df_pop_nc16 = pd.merge(nc_df_2016,pop_combined[['County', 2016]],on="County",how='left')
df_pop_nc17 = pd.merge(nc_df_2017,pop_combined[['County', 2017]],on="County",how='left')
df_pop_nc18 = pd.merge(nc_df_2018,pop_combined[['County', 2018]],on="County",how='left')
df_pop_nc19 = pd.merge(nc_df_2019,pop_combined[['County', 2019]],on="County",how='left')
df_pop_nc20 = pd.merge(nc_df_2020,pop_combined[['County', 2020]],on="County",how='left')
df_comb_nc16 = pd.merge(df_pop_nc16,metros_nc[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_comb_nc17 = pd.merge(df_pop_nc17,metros_nc[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_comb_nc18 = pd.merge(df_pop_nc18,metros_nc[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_comb_nc19 = pd.merge(df_pop_nc19,metros_nc[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_comb_nc20 = pd.merge(df_pop_nc20,metros_nc[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_comb_nc16.rename(columns = {2016:'population'}, inplace = True)
df_comb_nc16.insert(0,'Year',2016)
df_comb_nc17.rename(columns = {2017:'population'}, inplace = True)
df_comb_nc17.insert(0,'Year',2017)
df_comb_nc18.rename(columns = {2018:'population'}, inplace = True)
df_comb_nc18.insert(0,'Year',2018)
df_comb_nc19.rename(columns = {2019:'population'}, inplace = True)
df_comb_nc19.insert(0,'Year',2019)
df_comb_nc20.rename(columns = {2020:'population'}, inplace = True)
df_comb_nc20.insert(0,'Year',2020)
df_comb_nc16.to_excel('../data/cleaned_data/2016_NC.xlsx', engine='xlsxwriter',index=False)
df_comb_nc17.to_excel('../data/cleaned_data/2017_NC.xlsx', engine='xlsxwriter',index=False)
df_comb_nc18.to_excel('../data/cleaned_data/2018_NC.xlsx', engine='xlsxwriter',index=False)
df_comb_nc19.to_excel('../data/cleaned_data/2019_NC.xlsx', engine='xlsxwriter',index=False)
df_comb_nc20.to_excel('../data/cleaned_data/2020_NC.xlsx', engine='xlsxwriter',index=False)

### Combining all the files into a master file

In [19]:
data_file_folder ='../data/cleaned_data/'
df = []
for file in os.listdir(data_file_folder):
        if file.endswith('_NC.xlsx'):             
             df.append(pd.read_excel(os.path.join(data_file_folder,file),sheet_name='Sheet1'))
df_nc_master = pd.concat(df, axis = 0)

In [20]:
df_nc_master.isna().sum()

Year                                            0
County                                          0
fatalities                                      0
injuries                                        0
total_crashes                                   0
population                                      0
CBSA Title                                    200
Metropolitan/Micropolitan Statistical Area    200
dtype: int64

In [21]:
df_nc_master.insert(1,'State','North Carolina') 
df_nc_master.head(10)

,Year,State,County,fatalities,injuries,total_crashes,population,CBSA Title,Metropolitan/Micropolitan Statistical Area
0,2013,North Carolina,ALAMANCE,23,1754,3506,154162,"Burlington, NC",Metropolitan Statistical Area
1,2013,North Carolina,ALEXANDER,8,212,455,37012,"Hickory-Lenoir-Morganton, NC",Metropolitan Statistical Area
2,2013,North Carolina,ALLEGHANY,3,98,184,10903,NaN,NaN
3,2013,North Carolina,ANSON,7,370,597,25957,"Charlotte-Concord-Gastonia, NC-SC",Metropolitan Statistical Area
4,2013,North Carolina,ASHE,4,206,431,26840,NaN,NaN
5,2013,North Carolina,AVERY,2,151,289,17654,NaN,NaN
6,2013,North Carolina,BEAUFORT,13,435,961,47382,"Washington, NC",Micropolitan Statistical Area
7,2013,North Carolina,BERTIE,5,251,431,20326,NaN,NaN
8,2013,North Carolina,BLADEN,16,499,807,34618,NaN,NaN
9,2013,North Carolina,BRUNSWICK,13,904,2111,114922,"Myrtle Beach-Conway-North Myrtle Beach, SC-NC",Metropolitan Statistical Area


In [22]:
df_nc_master.tail(10)

,Year,State,County,fatalities,injuries,total_crashes,population,CBSA Title,Metropolitan/Micropolitan Statistical Area
90,2020,North Carolina,VANCE,14,611,616,42529,"Henderson, NC",Micropolitan Statistical Area
91,2020,North Carolina,WAKE,84,"10,130","13,386",1133553,"Raleigh-Cary, NC",Metropolitan Statistical Area
92,2020,North Carolina,WARREN,10,179,231,18598,NaN,NaN
93,2020,North Carolina,WASHINGTON,0,63,100,10955,NaN,NaN
94,2020,North Carolina,WATAUGA,6,341,464,54072,"Boone, NC",Micropolitan Statistical Area
95,2020,North Carolina,WAYNE,25,"1,272","1,594",117226,"Goldsboro, NC",Metropolitan Statistical Area
96,2020,North Carolina,WILKES,13,630,630,65871,"North Wilkesboro, NC",Micropolitan Statistical Area
97,2020,North Carolina,WILSON,18,"1,170","1,236",78811,"Wilson, NC",Micropolitan Statistical Area
98,2020,North Carolina,YADKIN,11,266,295,37182,"Winston-Salem, NC",Metropolitan Statistical Area
99,2020,North Carolina,YANCEY,2,98,104,18475,NaN,NaN


In [23]:
df_nc_master.to_excel('../data/NC.xlsx', engine='xlsxwriter',index=False)

In [24]:
df_nc_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 800 entries, 0 to 99
Data columns (total 9 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   Year                                        800 non-null    int64 
 1   State                                       800 non-null    object
 2   County                                      800 non-null    object
 3   fatalities                                  800 non-null    int64 
 4   injuries                                    800 non-null    object
 5   total_crashes                               800 non-null    object
 6   population                                  800 non-null    int64 
 7   CBSA Title                                  600 non-null    object
 8   Metropolitan/Micropolitan Statistical Area  600 non-null    object
dtypes: int64(3), object(6)
memory usage: 62.5+ KB
